# Humanoid Dynamics Concepts

## 1 基本模型
状态定义：全状态$q=[r, \phi, \theta]^T$，$r \in R^3$是三轴位移向量，$\phi \in R^3$是base的欧拉角向量，$\theta \in R^n$是各关节的角度。
动力学方程：$$\begin{align}I(q)\ddot{q}+C(q,\dot{q})+G(q)  &= u+E(q)^T \lambda \\ E(q) \dot{q} &= 0 \end{align}$$
其中$I(q)$是广义质量矩阵，$C(q,\dot{q})$是科里奥利项，$G(q)$是重力项，$u \in R^{6+n}$是控制项，$f_p$是约束力，$E(q)$是约束力的雅可比矩阵。约束项在不同的任务阶段会不同。接触约束可以通过多个点的接触方程组，或者两点接触方程+两点向量的方向约束（水平向的约束力和旋转力矩）。


## 2 basic concepts
### 质心动力学（Centroidal Dynamics）
bipea robot的inertial movement可以分为平动和转动两部分，分别对应动量守恒和角动量守恒。角动量是质心关于足端接触点产生的角动量。
### 零空间
进行任务分配，比如只动手。矩阵$H$的零空间定义为$$I - pinv(H) H$$
### contact mechanics
contact wrench cone:摩擦力锥 

contact wrench sum: 摩擦力矩集
### zero-moment point
center of pressure/zero-moment point：承压中心在承压点所包围区域内部。
